In [4]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

2023-11-08 16:52:30.050494: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 16:52:30.350336: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 16:52:30.352259: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 16:52:31.498918: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
# 데이터셋 경로 설정
base_dir = './DogEmotion'
emotions = ['happy', 'sad', 'relaxed', 'angry']

In [5]:
# # Train, Validation 데이터셋 분리
# train_datagen = ImageDataGenerator(
#     validation_split=0.2,  # 20%를 검증 데이터로 사용
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# train_generator = train_datagen.flow_from_directory(
#     base_dir,
#     target_size=(299, 299),
#     batch_size=16,
#     class_mode='categorical',
#     subset='training'  # 학습 데이터 부분
# )

# validation_generator = train_datagen.flow_from_directory(
#     base_dir,
#     target_size=(299, 299),
#     batch_size=16,
#     class_mode='categorical',
#     subset='validation'  # 검증 데이터 부분
# )

# InceptionResNetV2 모델
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

# 새로운 출력 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# 모델 구조 확인
# model.summary()

def print_layers_with_index(base_model):
    for i, layer in enumerate(base_model.layers):
        print(f"Index: {i}, Layer Name: {layer.name}")

print_layers_with_index(base_model)

219055592/219055592 [==============================] - 22s 0us/step
Index: 0, Layer Name: input_1
Index: 1, Layer Name: conv2d
Index: 2, Layer Name: batch_normalization
Index: 3, Layer Name: activation
Index: 4, Layer Name: conv2d_1
Index: 5, Layer Name: batch_normalization_1
Index: 6, Layer Name: activation_1
Index: 7, Layer Name: conv2d_2
Index: 8, Layer Name: batch_normalization_2
Index: 9, Layer Name: activation_2
Index: 10, Layer Name: max_pooling2d
Index: 11, Layer Name: conv2d_3
Index: 12, Layer Name: batch_normalization_3
Index: 13, Layer Name: activation_3
Index: 14, Layer Name: conv2d_4
Index: 15, Layer Name: batch_normalization_4
Index: 16, Layer Name: activation_4
Index: 17, Layer Name: max_pooling2d_1
Index: 18, Layer Name: conv2d_8
Index: 19, Layer Name: batch_normalization_8
Index: 20, Layer Name: activation_8
Index: 21, Layer Name: conv2d_6
Index: 22, Layer Name: conv2d_9
Index: 23, Layer Name: batch_normalization_6
Index: 24, Layer Name: batch_normalization_9
Index: 25

In [23]:
# InceptionResNetV2 모델 층들을 동결
for layer in base_model.layers:
    layer.trainable = False

# 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50
)

# InceptionResNetV2의 모델 층들을 동결 해제
for layer in base_model.layers:
    layer.trainable = True

# 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50)

# 모델 평가
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Epoch 1/50


2023-11-06 14:21:04.280146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2023-11-06 14:21:05.336083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-06 14:21:07.576796: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5650a79aab80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-06 14:21:07.576856: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2023-11-06 14:21:07.620943: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


  1/200 [..............................] - ETA: 42:17 - loss: 1.3770 - accuracy: 0.3125

2023-11-06 14:21:07.865198: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


200/200 [==============================] - 72s 298ms/step - loss: 1.1779 - accuracy: 0.4806 - val_loss: 1.1250 - val_accuracy: 0.5113
Epoch 2/50
200/200 [==============================] - 56s 278ms/step - loss: 1.0227 - accuracy: 0.5809 - val_loss: 1.0611 - val_accuracy: 0.5500
Epoch 3/50
200/200 [==============================] - 56s 281ms/step - loss: 0.9496 - accuracy: 0.6150 - val_loss: 0.9975 - val_accuracy: 0.5813
Epoch 4/50
200/200 [==============================] - 56s 282ms/step - loss: 0.9103 - accuracy: 0.6394 - val_loss: 0.9855 - val_accuracy: 0.5888
Epoch 5/50
200/200 [==============================] - 57s 284ms/step - loss: 0.8828 - accuracy: 0.6409 - val_loss: 0.9605 - val_accuracy: 0.6212
Epoch 6/50
200/200 [==============================] - 57s 283ms/step - loss: 0.8606 - accuracy: 0.6634 - val_loss: 0.9714 - val_accuracy: 0.5975
Epoch 7/50
200/200 [==============================] - 56s 281ms/step - loss: 0.8451 - accuracy: 0.6584 - val_loss: 0.9722 - val_accuracy: 0.5

2023-11-06 15:07:53.035855: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 144.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-06 15:07:53.362601: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 144.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-06 15:07:53.453880: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 490.41MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-06 15:07:53.634143: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocato

200/200 [==============================] - 143s 398ms/step - loss: 0.6735 - accuracy: 0.7325 - val_loss: 0.7401 - val_accuracy: 0.7100
Epoch 2/50
200/200 [==============================] - 78s 388ms/step - loss: 0.4600 - accuracy: 0.8275 - val_loss: 0.7700 - val_accuracy: 0.7312
Epoch 3/50
200/200 [==============================] - 78s 389ms/step - loss: 0.3686 - accuracy: 0.8647 - val_loss: 0.7108 - val_accuracy: 0.7912
Epoch 4/50
200/200 [==============================] - 78s 388ms/step - loss: 0.2925 - accuracy: 0.8944 - val_loss: 0.5971 - val_accuracy: 0.8025
Epoch 5/50
200/200 [==============================] - 78s 388ms/step - loss: 0.2316 - accuracy: 0.9194 - val_loss: 0.5331 - val_accuracy: 0.8250
Epoch 6/50
200/200 [==============================] - 78s 388ms/step - loss: 0.2001 - accuracy: 0.9319 - val_loss: 0.6016 - val_accuracy: 0.8138
Epoch 7/50
200/200 [==============================] - 78s 388ms/step - loss: 0.1627 - accuracy: 0.9478 - val_loss: 0.6372 - val_accuracy: 0.